In [1]:
import transformers
import json
import sys
import os
import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


In [2]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, lr, dataset_name, adapter_checkpoint):
    !python run_qa_alt2.py \
      --model_name_or_path {checkpoint} \
      --dataset_name {dataset_name} \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 32\
      --per_device_eval_batch_size 32\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate {lr} \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir \
      --train_adapter \
      --adapter_config houlsby \
      --load_adapter {adapter_checkpoint}

In [3]:
data_files = {}
data_files["train"] = covid_file

covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)

squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

# squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
# covid_qa = datasets.Dataset.from_dict(covid_qa[:20])
# bio_qa = datasets.Dataset.from_dict(bio_qa[:20])

Using custom data configuration default-a396986d6b3a2375
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-a396986d6b3a2375\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-8fdbe041288a2f4d
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-8fdbe041288a2f4d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [ ]:
def domain_pruning(lr):
    k_fold = 5
    K = 6

    for i in range(k_fold):
        covid_fold = covid_qa.shard(k_fold, i)

        covid_test = covid_fold.shard(2, 0)
        covid_val = covid_fold.shard(2, 1)
        covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

        #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

        checkpoint = 'roberta-base'
        adapter_load = '@ukp/roberta-base_qa_squad1_houlsby'
        cur_dir = '../adapter_models/gradual_ft_baseline_lr'+str(lr)+'/split_' + str(i)

        #log_file = open(cur_dir + '/log_file.txt',"w+")

        #sys.stdout = log_file

        squad_qa.shuffle()
        bio_qa.shuffle()

        num_of_shards = int(K/2)
        squad_qa_shards = [squad_qa.shard(num_of_shards,i) for i in range(num_of_shards)]
        bio_qa_shards = [bio_qa.shard(num_of_shards,i) for i in range(num_of_shards)]

        squad_qa_cur = concatenate_datasets(squad_qa_shards)
        bio_qa_cur = concatenate_datasets(bio_qa_shards)

        squad_rows_to_remove = len(squad_qa_shards[0])
        bio_rows_to_remove = len(bio_qa_shards[0])
        L = squad_qa_shards + bio_qa_shards + [covid_train]
        print(f'Gradual finetuning with K={K} ({K//2} per SQuAD and bioASQ)')
        for n in range(K+1):
            output_dir = cur_dir + '/checkpoint_' + str(n)


            full_dataset = concatenate_datasets(L)
            make_and_save_full_dataset(full_dataset, covid_val, covid_test, covid_and_squad_dataset_path)

            print(f'n={n} =============')
            print('Total: ',full_dataset.num_rows)

            run_gradual_ft(output_dir, checkpoint, lr, covid_and_squad_dataset_path, adapter_load)
            print('=================\n')

            checkpoint = output_dir
            adapter_load = checkpoint + '/full_squad_covidQA'

            if n < K//2:
                print(f'Removing {squad_rows_to_remove} from SQuAD')
                _ = L.pop(0)

            elif n < K:
                print(f'Removing {bio_rows_to_remove} from bioASQ')
                _ = L.pop(0)

            else:
                assert full_dataset.num_rows == covid_train.num_rows
                full_dataset = covid_train

        #log_file.close()
    #sys.stdout = sys.__stdout__


Loading cached shuffled indices for dataset at C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16\cache-30b2dcc23b00412a.arrow


Gradual finetuning with K=6 (3 per SQuAD and bioASQ)
n=0 =============
Total:  112676
08/06/2021 00:11:58 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/06/2021 00:11:58 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,

Removing 32723 from SQuADgradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,

ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_mo

[INFO|tokenization_auto.py:427] 2021-08-06 00:11:59,311 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:531] 2021-08-06 00:11:59,658 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:569] 2021-08-06 00:11:59,658 >> Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings":

n=1 =============
Total:  79953
08/06/2021 00:43:53 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/06/2021 00:43:53 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_

[INFO|tokenization_utils_base.py:1651] 2021-08-06 00:43:53,869 >> Didn't find file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_0\added_tokens.json. We won't load it.
[INFO|tokenization_utils_base.py:1715] 2021-08-06 00:43:53,870 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_0\vocab.json
[INFO|tokenization_utils_base.py:1715] 2021-08-06 00:43:53,870 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_0\merges.txt
[INFO|tokenization_utils_base.py:1715] 2021-08-06 00:43:53,870 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_0\tokenizer.json
[INFO|tokenization_utils_base.py:1715] 2021-08-06 00:43:53,870 >> loading file None
[INFO|tokenization_utils_base.py:1715] 2021-08-06 00:43:53,870 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_0\special_tokens_map.json
[INFO|tokenization_utils_base.py:1715] 2021-08-06 00:43:53,870 >> loading file ../adapter

...

08/06/2021 01:55:33 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/06/2021 01:55:33 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,=================


eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
Removing 4298 from bioASQfp16_full_eval=False,

fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,

100%|##########| 191/191 [00:28<00:00,  7.46it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

  1%|          | 2/202 [00:00<00:20,  9.57it/s]

  3%|2         | 6/202 [00:00<00:16, 12.08it/s]

  4%|3         | 8/202 [00:00<00:15, 12.18it/s]

  5%|4         | 10/202 [00:00<00:14, 13.39it/s]

  6%|5         | 12/202 [00:00<00:13, 14.29it/s]

  7%|6         | 14/202 [00:00<00:12, 14.79it/s]

  8%|8         | 17/202 [00:01<00:11, 15.74it/s]

  9%|9         | 19/202 [00:01<00:11, 16.20it/s]

 11%|#         | 22/202 [00:01<00:10, 17.45it/s]

 12%|#1        | 24/202 [00:01<00:10, 16.53it/s]

 13%|#3        | 27/202 [00:01<00:09, 17.84it/s]

 15%|#4        | 30/202 [00:01<00:08, 19.79it/s]

 16%|#6        | 33/202 [00:01<00:09, 17.40it/s]

 17%|#7        | 35/202 [00:02<00:10, 16.25it/s]

 18%|#8        | 37/202 [00:02<00:10, 15.08it/s]

 20%|#9        | 40/202 [00:02<00:09, 16.49it/s]

 21%|##        | 42/202 [00:02<00:09, 16.80it/s]

 22%|##1       | 44/202 [00:02<00:10, 14.89it/s]

 23%|##2   

n=6 =============
Total:  1615
08/06/2021 02:07:28 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/06/2021 02:07:28 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_n

[INFO|tokenization_utils_base.py:1651] 2021-08-06 02:07:28,594 >> Didn't find file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_5\added_tokens.json. We won't load it.
[INFO|tokenization_utils_base.py:1715] 2021-08-06 02:07:28,595 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_5\vocab.json
[INFO|tokenization_utils_base.py:1715] 2021-08-06 02:07:28,595 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_5\merges.txt
[INFO|tokenization_utils_base.py:1715] 2021-08-06 02:07:28,595 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_5\tokenizer.json
[INFO|tokenization_utils_base.py:1715] 2021-08-06 02:07:28,595 >> loading file None
[INFO|tokenization_utils_base.py:1715] 2021-08-06 02:07:28,595 >> loading file ../adapter_models/gradual_ft_baseline_lr1e-5/split_0/checkpoint_5\special_tokens_map.json
[INFO|tokenization_utils_base.py:1715] 2021-08-06 02:07:28,595 >> loading file ../adapter

Gradual finetuning with K=6 (3 per SQuAD and bioASQ)
n=0 =============
Total:  112676


In [ ]:
lr = 3e-5
domain_pruning(lr)

